# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,luanda,-8.8368,13.2343,76.77,87,100,7.29,AO,1643591748
1,1,bichena,10.4500,38.2000,53.44,47,91,1.83,ET,1643592039
2,2,porbandar,21.6422,69.6093,66.34,85,0,7.16,IN,1643592039
3,3,saskylakh,71.9167,114.0833,-23.13,100,94,2.82,RU,1643592040
4,4,ayr,55.4627,-4.6339,42.78,70,75,36.82,GB,1643592040


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#config gmaps 
gmaps.configure(api_key=g_key)


In [4]:
locations = city_data_df[["Lat","Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)




### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
 idealWeatherDF = city_data_df.loc[
     (city_data_df["Max Temp"] > 70) & (city_data_df["Max Temp"] < 80) & \
     (city_data_df["Wind Speed"] < 10) & \
     (city_data_df["Cloudiness"] == 0)
].dropna()
    
idealWeatherDF.head()
    

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
144,144,concordia,-31.3930,-58.0209,78.78,52,0,4.00,AR,1643591989
218,218,itaqui,-29.1253,-56.5531,76.66,35,0,7.85,BR,1643592091
243,243,cayenne,4.9333,-52.3333,79.09,85,0,1.99,GF,1643592234
292,292,sainte-suzanne,-20.9061,55.6089,75.42,73,0,8.05,RE,1643592274


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = idealWeatherDF[["City","Country",'Lat', 'Lng']]
hotel_df['Hotel Name'] = ''
hotel_df

/var/folders/sb/5qx_22bj46lcynzqfkqv8mkc0000gn/T/ipykernel_3267/368109666.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Country,Lat,Lng,Hotel Name
144,concordia,AR,-31.3930,-58.0209,
218,itaqui,BR,-29.1253,-56.5531,
243,cayenne,GF,4.9333,-52.3333,
292,sainte-suzanne,RE,-20.9061,55.6089,


In [8]:
params = {
    'radius':5000,
    'types': 'lodging',
    'key': g_key
}

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat},{lng}"
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    name_address = requests.get(base_url, params =params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]        
    except:
        print(f"Error finding lodging in {row['City']}. Skipping...")
        
hotel_df    

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,City,Country,Lat,Lng,Hotel Name
144,concordia,AR,-31.3930,-58.0209,Palmar Hotel Casino
218,itaqui,BR,-29.1253,-56.5531,HOTEL BIASI - ITAQUI/RS
243,cayenne,GF,4.9333,-52.3333,Hôtel Le Dronmi
292,sainte-suzanne,RE,-20.9061,55.6089,Gites Ango


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig